In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from random import seed, shuffle
import os,sys
import traceback
import numpy as np
from random import seed, shuffle
from collections import defaultdict
from copy import deepcopy
from cvxpy import *
import cvxpy as cp
import numpy as np
import time


In [141]:
!pip3 install --upgrade pip
!pip install setuptools==57.5.0 #adde
!pip3 install dccp

  Using cached setuptools-57.5.0-py3-none-any.whl (819 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 69.0.2
    Uninstalling setuptools-69.0.2:
      Successfully uninstalled setuptools-69.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
arviz 0.15.1 requires setuptools>=60.0.0, but you have setuptools 57.5.0 which is incompatible.
cvxpy 1.3.2 requires setuptools>65.5.1, but you have setuptools 57.5.0 which is incompatible.


  Using cached setuptools-69.0.2-py3-none-any.whl.metadata (6.3 kB)
Using cached setuptools-69.0.2-py3-none-any.whl (819 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.5.0
    Uninstalling setuptools-57.5.0:
      Successfully uninstalled setuptools-57.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [21]:
# Load data
df = pd.read_csv('https://github.com/propublica/compas-analysis/raw/master/compas-scores-two-years.csv')


In [22]:
# Data Preprocessing
df['day_stayed'] = (pd.to_datetime(df["c_jail_out"]) - pd.to_datetime(df["c_jail_in"])).astype('timedelta64[h]') / 24
df = df[df.day_stayed > 0]

# Select relevant variables
df = df[['sex', 'age_cat', 'race', 'decile_score', 'score_text', 'priors_count', 'days_b_screening_arrest',
         'v_decile_score', 'v_score_text', 'c_charge_degree', 'is_violent_recid', 'is_recid', 'day_stayed', 'two_year_recid']]

# Filter for two races
df = df[df.race.isin(['African-American', 'Caucasian'])]

# Encoding categorical variables
df['sex'].replace(['Male', 'Female'], [1, 0], inplace=True)
df['race'].replace(['Caucasian', 'African-American'], [1, 0], inplace=True)
df['score_text'].replace(['Low', 'Medium', 'High'], [0, 1, 2], inplace=True)
df['v_score_text'].replace(['Low', 'Medium', 'High'], [0, 1, 2], inplace=True)
df['age_cat'].replace(['Less than 25', '25 - 45', 'Greater than 45'], [0, 1, 2], inplace=True)
df['c_charge_degree'].replace(['M', 'F'], [1, 0], inplace=True)


In [23]:
# Assuming 'df' is your DataFrame with features and labels
X = df.drop('two_year_recid', axis=1).values
y = df['two_year_recid'].values
y = np.where(y == 0, -1, 1)  # Convert labels to {-1, 1}
sensitive_attr = df['race'].values

# Adding an intercept term to X
intercept = np.ones((X.shape[0], 1))  # Column of ones
X = np.hstack((intercept, X))

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test, sensitive_attr_train, sensitive_attr_test = train_test_split(
    X, y, sensitive_attr, test_size=0.2, random_state=42)


In [52]:
!pip3 install dccp

**Building Fairness Constraints:** The get_constraint_list_cov function builds the fairness constraints based on the sensitive attribute and the thresholds defined in sensitive_attrs_to_cov_thresh. These constraints aim to ensure that the difference in treatment of different groups (based on the sensitive attribute) does not exceed specified thresholds.

In [63]:
def calculate_fairness_covariance(theta, features, true_labels, sensitive_features):
    assert(features.shape[0] == sensitive_features.shape[0])
    if len(set(sensitive_features)) != 2:
        raise Exception("Non-binary attribute detected. This function handles binary attributes only.")

    predicted_labels = np.sign(np.dot(features, theta))

    group_totals = {ct: {} for ct in [0, 1, 2]}
    group_averages = {ct: {} for ct in [0, 1, 2]}
    covariance_sums = {ct: {} for ct in [0, 1, 2]}

    for group_val in set(sensitive_features):
        group_totals[0][group_val] = np.sum(sensitive_features == group_val)
        group_totals[1][group_val] = np.sum(np.logical_and(sensitive_features == group_val, true_labels == -1))
        group_totals[2][group_val] = np.sum(np.logical_and(sensitive_features == group_val, true_labels == 1))

    for ct in [0, 1, 2]:
        for group_val in set(sensitive_features):
            total = group_totals[ct].get(group_val, 0) + group_totals[ct].get(1-group_val, 0)
            if total > 0:
                group_averages[ct][group_val] = group_totals[ct].get(group_val, 0) / float(total)
            else:
                group_averages[ct][group_val] = 0

    label_product = np.array(predicted_labels * true_labels)
    for group_val in set(sensitive_features):
        idx = sensitive_features == group_val
        dist_bound_prod = label_product[idx]
        for ct in [0, 1, 2]:
            if ct == 0:
                covariance_sums[ct][group_val] = np.sum(np.minimum(0, dist_bound_prod)) * (group_averages[ct][group_val] / len(features))
            elif ct == 1:
                covariance_sums[ct][group_val] = np.sum(np.minimum(0, ((1 - true_labels[idx]) / 2) * dist_bound_prod)) * (group_averages[ct][group_val] / np.sum(true_labels == -1))
            else:
                covariance_sums[ct][group_val] = np.sum(np.minimum(0, ((1 + true_labels[idx]) / 2) * dist_bound_prod)) * (group_averages[ct][group_val] / np.sum(true_labels == 1))

    return covariance_sums



def predict(X, theta):
    # Using numpy's dot product for matrix multiplication
    return np.sign(np.dot(X, theta))




In [64]:
# Define constraint parameters
constraint_params = {
    "sensitive_attrs_to_cov_thresh": {
        1: 0.05,  # Threshold for FPR
        2: 0.05   # Threshold for FNR
    },
    "cons_type": 4,  # Indicates both FPR and FNR constraints are applied
    "tau": 0.005,    # DCCP method parameter
    "mu": 1.2        # DCCP method parameter
}
# Train the model with fairness constraints
trained_theta = train_model(X_train, y_train, sensitive_attr_train, eps=1e-5, constraint_params=constraint_params)

# Evaluate the trained model using get_clf_stats
train_accuracy, test_accuracy, train_fair_cov, test_fair_cov, train_fair_metrics, test_fair_metrics = get_clf_stats(
    trained_theta, X_train, y_train, sensitive_attr_train, X_test, y_test, sensitive_attr_test
)

# Output the results
print(f"Training Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")
print("Training Fairness Metrics:", train_fair_metrics)
print("Test Fairness Metrics:", test_fair_metrics)


Test Accuracy: 0.9692982456140351
Training Accuracy: 0.9681947795569203
Test Accuracy: 0.9692982456140351
Training Fairness Metrics: {0: {'fp': 100, 'fn': 0, 'fpr': 0.07776049766718507, 'fnr': 0.0, 'acc': 0.9635435654393001}, 1: {'fp': 45, 'fn': 0, 'fpr': 0.041743970315398886, 'fnr': 0.0, 'acc': 0.9752202643171806}}
Test Fairness Metrics: {0: {'fp': 26, 'fn': 0, 'fpr': 0.08333333333333333, 'fnr': 0.0, 'acc': 0.9614243323442137}, 1: {'fp': 9, 'fn': 0, 'fpr': 0.032490974729241874, 'fnr': 0.0, 'acc': 0.98068669527897}}


use get_constraint_list_cov in logistic regression model training process, it adds constraints to the optimization problem to control the FPR and FNR for different groups based on the sensitive attribute